# Reproducing Effect-Grounded Lean Tactics (EGLT)

This notebook runs the end-to-end pipeline:
1. Load configs
2. Preprocess StepRecord JSONL → DeltaRecord JSONL
3. Train (SGNS / CBOW-Δ / Seq baseline)
4. Evaluate retrieval (MRR / Recall@k), analogy, and produce UMAP Figure 1

**Assumptions**
- Run from repository root.
- `data/raw/steps.jsonl` exists (StepRecord JSONL).
- Configs live in `configs/`.

> Tip: For quick sanity checks, run `python scripts/smoke_test.py`.

In [2]:

import os, sys
from pathlib import Path

# --- Resolve repo root and import eglT package from src/ ---
REPO_ROOT = Path.cwd().resolve()
if not ((REPO_ROOT / "pyproject.toml").exists() and (REPO_ROOT / "src").exists()):
    if (REPO_ROOT.parent / "pyproject.toml").exists():
        REPO_ROOT = REPO_ROOT.parent.resolve()

SRC = REPO_ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print("REPO_ROOT:", REPO_ROOT)
print("SRC:", SRC)


REPO_ROOT: C:\Users\AsusPro\Desktop\Modeling-Tactics-as-Operators
SRC: C:\Users\AsusPro\Desktop\Modeling-Tactics-as-Operators\src


In [3]:

import yaml
from pathlib import Path

cfg_dir = REPO_ROOT / "configs"

dataset_cfg = yaml.safe_load((cfg_dir / "dataset.yaml").read_text(encoding="utf-8"))
preprocess_cfg = yaml.safe_load((cfg_dir / "preprocess.yaml").read_text(encoding="utf-8"))
train_sgns_cfg = yaml.safe_load((cfg_dir / "train_sgns.yaml").read_text(encoding="utf-8"))
train_cbow_cfg = yaml.safe_load((cfg_dir / "train_cbow.yaml").read_text(encoding="utf-8"))
train_seq_cfg  = yaml.safe_load((cfg_dir / "train_seq.yaml").read_text(encoding="utf-8"))
eval_cfg = yaml.safe_load((cfg_dir / "eval.yaml").read_text(encoding="utf-8"))

print("Loaded configs from:", cfg_dir)
print("raw_jsonl:", dataset_cfg.get("raw_jsonl"))
print("out_deltas_jsonl:", dataset_cfg.get("out_deltas_jsonl"))


Loaded configs from: C:\Users\AsusPro\Desktop\Modeling-Tactics-as-Operators\configs
raw_jsonl: data/raw/steps.jsonl
out_deltas_jsonl: data/processed/deltas.jsonl


## Dataset sanity checks

Compute tactic frequencies and basic integrity checks over the raw StepRecord JSONL.

In [4]:

from eglt.dataset.load_raw import load_step_records_jsonl
from eglt.dataset.stats import compute_stats, format_report
from eglt.paths import paths as eglt_paths

P = eglt_paths(REPO_ROOT)
raw_path = (P.root / dataset_cfg["raw_jsonl"]).resolve()

recs = list(load_step_records_jsonl(raw_path))
rep = compute_stats(recs)
print(format_report(rep, top_k=30))


Records: 321554
Unique proofs: 76343

Missing fields:
  proof_id: 0
  step_id: 0
  tactic_head: 0

Empty/None token fields:
  state_before_tokens: 321554
  state_after_tokens: 321554

Top-30 tactics:
  rw	59086
  simp	43472
  exact	37355
  have	21161
  refine	17188
  apply	12629
  simpa	10586
  obtain	10436
  intro	9967
  ext	6883
  rcases	6553
  simp_rw	6090
  rintro	5783
  let	4383
  rfl	4112
  rwa	3750
  by_cases	3521
  convert	3016
  cases	3013
  dsimp	2911
  constructor	2744
  induction	2011
  congr	1986
  suffices	1961
  use	1737
  filter_upwards	1461
  haveI	1319
  omega	1315
  classical	1269
  calc	1204


## 1) Preprocess: StepRecord → DeltaRecord

Build `data/processed/deltas.jsonl` from the raw stepwise corpus.

In [5]:

from eglt.experiments.run_preprocess import run_preprocess
from pathlib import Path

deltas_path = run_preprocess(dataset_cfg, preprocess_cfg)
print("DeltaRecord JSONL:", deltas_path, "exists:", Path(deltas_path).exists())


[17:00:54] INFO     Reading raw JSONL: C:\Users\AsusPro\Desktop\Modeling-Tactics-as-Operators\data\raw\steps.jsonl

build_delta_context: 100%|██████████| 321554/321554 [02:38<00:00, 2022.39it/s]


[17:03:56] INFO     Wrote DeltaRecord JSONL:                                                                       
                    C:\Users\AsusPro\Desktop\Modeling-Tactics-as-Operators\data\processed\deltas.jsonl

DeltaRecord JSONL: C:\Users\AsusPro\Desktop\Modeling-Tactics-as-Operators\data\processed\deltas.jsonl exists: True


## 2) Train

Select one of: **SGNS**, **CBOW-Δ**, or **Seq baseline**.

By default, trains SGNS. Toggle flags below.

In [6]:
import sys
import site
print("sys.executable:", sys.executable)
print("site-packages dirs:", site.getsitepackages() if hasattr(site, "getsitepackages") else site.getusersitepackages())
try:
    import torch
    print("torch:", torch.__version__, "cuda:", torch.version.cuda, "cuda_available:", torch.cuda.is_available())
except Exception as e:
    print("torch import error:", e)

sys.executable: c:\Users\AsusPro\AppData\Local\Programs\Python\Python311\python.exe
site-packages dirs: ['c:\\Users\\AsusPro\\AppData\\Local\\Programs\\Python\\Python311', 'c:\\Users\\AsusPro\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages']
torch: 2.7.1+cu118 cuda: 11.8 cuda_available: True


In [7]:

from eglt.experiments.run_train import run_train
from eglt.paths import paths as eglt_paths
from pathlib import Path

P = eglt_paths(REPO_ROOT)

# ---- Choose what to train ----
DO_SGNS = True
DO_CBOW = False
DO_SEQ  = False

# ensure train configs point to latest deltas path
train_sgns_cfg["dataset_path"] = str(deltas_path)
train_cbow_cfg["dataset_path"] = str(deltas_path)

# If you want GPU and it's available, set in configs:
train_sgns_cfg["device"] = "cuda"
train_cbow_cfg["device"] = "cuda"

run_dirs = []

if DO_SGNS:
    run_dir = (P.results_runs / train_sgns_cfg.get("run_name", "sgns_delta")).resolve()
    run_train(train_sgns_cfg, run_dir)
    run_dirs.append(run_dir)

if DO_CBOW:
    run_dir = (P.results_runs / train_cbow_cfg.get("run_name", "cbow_delta")).resolve()
    run_train(train_cbow_cfg, run_dir)
    run_dirs.append(run_dir)

if DO_SEQ:
    train_seq_cfg.setdefault("step_records_path", dataset_cfg.get("raw_jsonl"))
    run_dir = (P.results_runs / train_seq_cfg.get("run_name", "seq_baseline")).resolve()
    run_train(train_seq_cfg, run_dir)
    run_dirs.append(run_dir)

print("Finished training runs:")
for d in run_dirs:
    print(" -", d)


[17:04:32] INFO     [SGNS] epoch=1 avg_loss=1.270185 pairs/s=148161.8

[17:04:45] INFO     [SGNS] epoch=2 avg_loss=1.658925 pairs/s=162443.7

[17:04:57] INFO     [SGNS] epoch=3 avg_loss=2.204746 pairs/s=159907.4

[17:05:10] INFO     [SGNS] epoch=4 avg_loss=2.186854 pairs/s=159614.0

[17:05:23] INFO     [SGNS] epoch=5 avg_loss=2.177976 pairs/s=159211.1

...

[17:07:09] INFO     [SGNS] epoch=13 avg_loss=1.755810 pairs/s=144586.1

[17:07:23] INFO     [SGNS] epoch=14 avg_loss=1.738535 pairs/s=143559.1

[17:07:37] INFO     [SGNS] epoch=15 avg_loss=1.756627 pairs/s=145422.0

[17:07:38] INFO     Trained SGNS.

Finished training runs:
 - C:\Users\AsusPro\Desktop\Modeling-Tactics-as-Operators\results\runs\sgns_delta


## 3) Evaluate

Runs retrieval/analogy/UMAP evaluations and writes outputs to the run directory and `results/figures/`.

In [ ]:

from eglt.experiments.run_eval import run_eval

eval_cfg["deltas_path"] = str(deltas_path)
eval_cfg['device'] = 'cpu'  # or 'cuda' if you want to use GPU for evaluation

for run_dir in run_dirs:
    run_eval(eval_cfg, run_dir)
    print("Eval done for:", run_dir)


[17:14:06] INFO     Retrieval: MRR=0.0734, n=321554

## 4) Inspect results

Load `eval_retrieval.yaml` and `eval_analogy.yaml` and display the UMAP figure.

In [ ]:

import yaml
from PIL import Image
import matplotlib.pyplot as plt
from eglt.paths import paths as eglt_paths
from pathlib import Path

P = eglt_paths(REPO_ROOT)

for run_dir in run_dirs:
    run_dir = Path(run_dir)
    print("\n=== RUN:", run_dir.name, "===\n")

    r_path = run_dir / "eval_retrieval.yaml"
    a_path = run_dir / "eval_analogy.yaml"

    if r_path.exists():
        ret = yaml.safe_load(r_path.read_text(encoding="utf-8"))
        print("Retrieval:", ret)
    else:
        print("Missing:", r_path)

    if a_path.exists():
        an = yaml.safe_load(a_path.read_text(encoding="utf-8"))
        print("Analogy:", an)
    else:
        print("Missing:", a_path)

    fig_path = P.results_figures / f"figure1_umap_{run_dir.name}.png"
    if fig_path.exists():
        img = Image.open(fig_path)
        plt.figure()
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"UMAP Figure 1 — {run_dir.name}")
        plt.show()
    else:
        print("Missing:", fig_path)


## One-shot reproduction

If you prefer a single command outside the notebook:

```bash
eglt-reproduce --dataset configs/dataset.yaml --preprocess configs/preprocess.yaml --train configs/train_sgns.yaml --eval configs/eval.yaml
```